In [1]:
import pandas as pd

In [2]:
df_review = pd.read_csv('IMDB_Dataset.csv')

In [3]:
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
df_review.columns

Index(['review', 'sentiment'], dtype='object')

In [5]:
df_positive = df_review[df_review["sentiment"] == "positive"][:9000]
df_negative = df_review[df_review["sentiment"] == "negative"][:1000]

In [6]:
df_review_imb = pd.concat([df_positive, df_negative])

In [8]:
from imblearn.under_sampling import  RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
df_review_bal, df_review_bal['sentiment']=rus.fit_resample(df_review_imb[['review']],df_review_imb['sentiment'])
df_review_bal

,review,sentiment
0,Basically there's a family where a little boy ...,negative
1,"This show was an amazing, fresh & innovative i...",negative
2,Encouraged by the positive comments about this...,negative
3,Phil the Alien is one of those quirky films wh...,negative
4,I saw this movie when I was about 12 when it c...,negative
...,...,...
1995,Knute Rockne led an extraordinary life and his...,positive
1996,At the height of the 'Celebrity Big Brother' r...,positive
1997,This is another of Robert Altman's underrated ...,positive
1998,This movie won a special award at Cannes for i...,positive


In [12]:
print(df_review_imb.value_counts("sentiment"))
print()
print(df_review_bal.value_counts("sentiment"))


sentiment
positive    9000
negative    1000
dtype: int64

sentiment
negative    1000
positive    1000
dtype: int64


In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [15]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)
train_x_vector

<1340x20625 sparse matrix of type '<class 'numpy.float64'>'
	with 118834 stored elements in Compressed Sparse Row format>

In [17]:
test_x_vector = tfidf.transform(test_x)

In [18]:
from sklearn.svm import SVC
svc = SVC(kernel="linear")
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

The code below sample a test using the SVC.predict function

In [19]:
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all'])))

['positive']
['positive']
['negative']


In [20]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

In [22]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(train_x_vector, train_y)

LogisticRegression()

In [30]:
# svc.score('Test samples', 'True labels')
print(f'SVC - {svc.score(test_x_vector, test_y)}')
print(f'DecisionTree - {dec_tree.score(test_x_vector, test_y)}')
print(f'Gaussian - {gnb.score(test_x_vector.toarray(), test_y)}')
print(f'Logistic Reg - {log_reg.score(test_x_vector, test_y)}')

SVC - 0.8409090909090909
DecisionTree - 0.6590909090909091
Gaussian - 0.6348484848484849
Logistic Reg - 0.8303030303030303


In [33]:
from sklearn.metrics import f1_score
f1_score(test_y, svc.predict(test_x_vector), labels=['positive', 'negative'], average=None)

array([0.84671533, 0.83464567])

In [34]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc.predict(test_x_vector), labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.83      0.87      0.85       335
    negative       0.85      0.82      0.83       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



In [36]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(test_y, svc.predict(test_x_vector), labels=['positive', 'negative'])
conf_mat

array([[290,  45],
       [ 60, 265]])

In [37]:
from sklearn.model_selection import GridSearchCV
#set the parameters
parameters = {'C': [1,4,8,16,32] ,'kernel':['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc,parameters, cv=5)

svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [38]:
print(svc_grid.best_params_)
print(svc_grid.best_estimator_)

{'C': 1, 'kernel': 'linear'}
SVC(C=1, kernel='linear')
